In [1]:
from sagemaker.sklearn.estimator import SKLearn
import pandas as pd
import numpy as np
import io
import boto3
import sagemaker
from sagemaker import get_execution_role, Session

In [2]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
bucket_name='sagemaker-us-east-1-729296914025'

In [3]:
data = "starbucks_imputed_scaled.csv"
data_dir = "data"
prefix = "affinity"
prefix_cluster = 'customers_clustered_aff'
path=f"{data_dir}/{data}"

In [4]:
sagepath=f"s3://sagemaker-us-east-1-729296914025/data/{data}"
print(sagepath)

s3://sagemaker-us-east-1-729296914025/data/starbucks_imputed_scaled.csv


In [5]:
empty_check = []
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    empty_check.append(obj.key)
    print(obj.key)

customers_clustered/kmeans-2021-12-22-14-21-19-823/output/model.tar.gz
customers_clustered/kmeans-2021-12-22-14-21-19-823/profiler-output/framework/training_job_end.ts
customers_clustered/kmeans-2021-12-22-14-21-19-823/profiler-output/system/incremental/2021122214/1640183040.algo-1.json
customers_clustered/kmeans-2021-12-22-14-21-19-823/profiler-output/system/incremental/2021122214/1640183100.algo-1.json
customers_clustered/kmeans-2021-12-22-14-21-19-823/profiler-output/system/training_job_end.ts
customers_clustered/kmeans-2021-12-22-14-42-50-791/output/model.tar.gz
customers_clustered/kmeans-2021-12-22-14-42-50-791/profiler-output/framework/training_job_end.ts
customers_clustered/kmeans-2021-12-22-14-42-50-791/profiler-output/system/incremental/2021122214/1640184360.algo-1.json
customers_clustered/kmeans-2021-12-22-14-42-50-791/profiler-output/system/incremental/2021122214/1640184420.algo-1.json
customers_clustered/kmeans-2021-12-22-14-42-50-791/profiler-output/system/training_job_end

In [6]:
#also measure where a customer resides before taking him any action, solely on demographics, thuis data that is not available
#but we want to let it run a few times so that the algorithm learns what the users has done after a while, to what he is sensified and to what he reacts so that he can decidie what to show next to generate more revenue

In [7]:
# your import and estimator code, here
Affinity = SKLearn(
    entry_point="train.py",
    source_dir="train_scripts",
    framework_version='0.23-1',
    role = role,
    train_instance_count=1,
    train_instance_type="ml.c5.2xlarge",
    sagemaker_session=sagemaker_session,
    hyperparameters = {'max_iter': 20000, 'random_state': 5, 'preference': -1500}
)
#other than 0

train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [8]:
%%time
#MemoryError: Unable to allocate 2.15 GiB for an array with shape (288796036,) and data type float64
#no convergence need random state 0
# Train estimator on S3 training data
Affinity.fit({
    "train": sagepath    
})


2022-03-27 16:16:51 Starting - Starting the training job...
2022-03-27 16:17:07 Starting - Preparing the instances for trainingProfilerReport-1648397811: InProgress
......
2022-03-27 16:18:20 Downloading - Downloading input data...
2022-03-27 16:18:43 Training - Downloading the training image...
2022-03-27 16:19:16 Training - Training image download completed. Training in progress..2022-03-27 16:19:19,044 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2022-03-27 16:19:19,046 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-03-27 16:19:19,054 sagemaker_sklearn_container.training INFO     Invoking user training script.
2022-03-27 16:19:19,463 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-03-27 16:19:19,474 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-03-27 16:19:19,483 sagemaker-training-toolkit INFO     No GPUs detected (norma

In [9]:
#fitted_affinity = "s3://sagemaker-us-east-1-729296914025/sagemaker-scikit-learn-2022-01-15-10-39-51-673/output/model.tar.gz" 

In [9]:
%%time

# deploy  model to create a predictor
predictor_aff = Affinity.deploy(
    initial_instance_count=1,
    instance_type='ml.c5.2xlarge'
)

------!CPU times: user 100 ms, sys: 17.9 ms, total: 118 ms
Wall time: 3min 1s


In [11]:
customers = pd.read_csv('s3://sagemaker-us-east-1-729296914025/data/starbucks_imputed_scaled.csv', delimiter=",") 
aws_conform = customers.values.astype("float32")

In [12]:
predictor_aff.predict(np.array(customers))

array([506, 492, 174, ..., 521, 568, 595])

In [ ]:
# we obviously got too many clusters. the preference parameter is something like the euclidean distance, 
# where negative values lead to less clusters..
# however, tests indicate that for example 5 clusters we need more than one day, making this approach not sufficient
# for more info please look at the notebook at the affinity excursus folder

# furthermore tuning the preference parameter is more or less the same as giving the cluster algorithm the amount of clusters..
# is there another way to get useful clusters?

In [13]:
customers

,time,value,gender,age,became_member_on,income,reward,offer_received,offer_viewed,offer_completed,...,last_time_offer_received,last_time_offer_completed,first_time_offer_received,first_time_offer_completed,mean_time_between_actions,amount_of_completed_user_paths,email,social,web,mobile
0,0.948956,0.183743,-0.758233,-1.290239,0.289966,0.404583,-0.842935,0.476725,0.465344,0.637901,...,0.464983,0.703987,-0.019422,0.0,-0.795001,-0.804229,0.697364,0.481531,0.632085,0.943784
1,0.176659,-0.198520,3.145252,-0.844766,1.119536,-0.312878,-0.207975,0.476725,1.236786,0.637901,...,-0.821974,0.703987,-0.019422,0.0,-0.146002,1.053214,1.023416,-0.468083,0.941412,0.303492
2,1.169612,0.733713,1.193510,0.364376,-0.576339,1.265537,0.290922,0.476725,-0.306098,0.637901,...,-0.821974,0.448149,-0.019422,0.0,-0.085252,0.124493,0.371312,1.431146,0.013433,0.623638
3,0.507643,0.393773,1.193510,-1.862990,-0.507236,-0.169386,-0.207975,-0.455228,0.465344,0.637901,...,-0.821974,0.797019,-0.019422,0.0,-0.730234,1.981936,0.371312,0.481531,0.322759,0.623638
4,0.948956,-0.445633,1.193510,-1.735712,0.307092,0.452414,-0.978997,0.476725,0.465344,0.637901,...,-0.821974,1.169147,-0.019422,0.0,-1.004075,-0.804229,0.697364,0.481531,0.632085,0.303492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16989,0.397315,-0.663763,-0.758233,-0.208375,0.333380,-0.632158,-0.434746,0.476725,1.236786,-1.230827,...,0.464983,-1.528780,-0.019422,0.0,-0.102445,-0.804229,0.045260,-1.417698,0.632085,-0.977091
16990,1.169612,-0.778268,-0.758233,0.109820,0.350249,-0.515681,1.198008,-0.455228,0.465344,-1.230827,...,-0.821974,-1.528780,-0.019422,0.0,2.105893,-0.804229,-0.606843,0.481531,-0.605219,-0.336799
16991,0.176659,-0.580704,-0.758233,-0.399292,-0.499359,-0.832241,-0.344038,-0.455228,-0.306098,-0.607918,...,0.464983,0.703987,-0.019422,0.0,-0.900674,0.124493,-0.606843,0.164993,-0.605219,-0.336799
16992,1.169612,-0.750316,-0.758233,0.046181,0.341857,-0.485109,-0.434746,0.476725,1.236786,-1.230827,...,-0.821974,-1.528780,-0.019422,0.0,1.038412,-0.804229,0.045260,-1.417698,0.013433,-0.977091
